In [1]:
import json
import pandas as pd
from datasets import Dataset
import os

file_path = "SAT_raw_predictions_gemini_2.5_flash_0.csv"

df = pd.read_csv(file_path)

SAT_DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT"
data = []

with open(os.path.join(SAT_DATASET_FOLDER, "SAT_labeled.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

ds = Dataset.from_list(data)

ds = ds.select(range(100))

FileNotFoundError: [Errno 2] No such file or directory: 'SAT_raw_predictions_gemini_2.5_flash_0.csv'

In [33]:
match_list = []

for i in range(len(df)):
    pred = df.iloc[i]["prediction"]
    if not isinstance(pred, str) or len(pred.strip()) == 0:
        match_list.append(False)
        continue

    pred_index = ord(pred) - ord('A')
    if pred_index < 0 or pred_index >= len(ds[i]["answer_choices"]):
        match_list.append(False)
        continue
    answer = ds[i]["answer_choices"][pred_index]
    if answer == ds[i]["correct_answer"]:
        match_list.append(True)
    else:
        match_list.append(False)

print(match_list)

[False, True, False, False, True, True, False, True, True, True, True, True, False, True, True, False, False, True, True, True, True, True, False, True, True, True, True, True, True, True, False, False, False, True, True, False, False, True, False, True, True, False, True, False, True, True, False, True, True, False, True, True, True, True, True, True, False, False, True, True, False, False, False, True, True, False, True, False, True, False, True, True, True, False, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, False, True, False, False, True, False, True, True, False]


In [34]:
# count the number of matches
matched = sum(match_list)
print(f"Matched {matched} out of {len(df)}")

# count the number of matches for each question
question_matches = {}


Matched 66 out of 100


In [3]:
print(df.head())

   index             category  \
0      0  DISTANCE_COMPARISON   
1      1    RELATIVE_LOCATION   
2      2    RELATIVE_LOCATION   
3      3    RELATIVE_LOCATION   
4      4    RELATIVE_LOCATION   

                                            question  \
0  Which point is closer to the camera taking thi...   
1  Considering the relative positions, is iron ch...   
2  Considering the relative positions, where is b...   
3  Considering the relative positions, is extenda...   
4  Considering the relative positions, where is s...   

                                           reasoning prediction answer  \
0  The image shows a perspective view where objec...          B      C   
1  The image shows a perspective view of a 3D sce...          A  below   
2  The image shows a single 3D object that appear...        NaN  right   
3  The image depicts a 3D scene with a vanishing ...          B  below   
4  The image shows a colorful landscape painting ...          A  right   

   is_correct      

In [ ]:
import json
import os

SAT_TRAIN_GENERATED_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT/train_generated"

In [12]:
global image_idx_counter
image_idx_counter = 0

from PIL import Image
from io import BytesIO
import base64

def base64_to_image(base64_str: str) -> Image.Image:
    """Convert a base64 string back to a PIL Image."""
    img_data = base64.b64decode(base64_str)
    return Image.open(BytesIO(img_data))

def deserialize_messages_to_image_paths(serialized):
    """Convert base64-encoded images in messages back to PIL Images."""
    global image_idx_counter
    deserialized = []
    for msg in serialized:
        new_msg = {"role": msg["role"], "content": []}
        for i, item in enumerate(msg["content"]):
            if item["type"] == "image" and "data" in item:
                # Reconstruct PIL image from base64 data
                image = base64_to_image(item["data"])
                image_path = f"{SAT_TRAIN_GENERATED_FOLDER}/images/image_{image_idx_counter}.png"
                image.save(image_path)
                image_idx_counter += 1
                new_msg["content"].append({
                    "type": "image",
                    "image": image_path
                })
                msg["content"][i + 1]["text"] = msg["content"][i + 1]["text"]
            else:
                new_msg["content"].append(item)
        deserialized.append(new_msg)
    return deserialized

def deserialize_messages_to_image(serialized):
    """Convert base64-encoded images in messages back to PIL Images."""
    global image_idx_counter
    deserialized = []
    for msg in serialized:
        new_msg = {"role": msg["role"], "content": []}
        for i, item in enumerate(msg["content"]):
            if item["type"] == "image" and "data" in item:
                # Reconstruct PIL image from base64 data
                image = base64_to_image(item["data"])
                new_msg["content"].append({
                    "type": "image",
                    "image": image
                })
            else:
                new_msg["content"].append(item)
        deserialized.append(new_msg)
    return deserialized

In [3]:
data = []
with open(os.path.join(SAT_TRAIN_GENERATED_FOLDER, "conversation_1.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

deserialized = deserialize_messages_to_image(data[0]["messages"])
print(deserialized)
# deserialized[0]["content"][0]["image"].save("test.png")

NameError: name 'SAT_TRAIN_GENERATED_FOLDER' is not defined

In [4]:
def parse_image_data(entry):
    processed_entry = {}
    deserialized = deserialize_messages_to_image_paths(entry["messages"])
    processed_entry["messages"] = deserialized
    processed_entry["response"] = entry["response"]
    processed_entry["conv_type"] = entry["conv_type"]
    return processed_entry


In [16]:
from datasets import Dataset
from tqdm import tqdm
import json
import os

SAT_DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT"
data = []

with open(os.path.join(SAT_DATASET_FOLDER, "SAT_labeled.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

ds = Dataset.from_list(data)

SAT_TRAIN_GENERATED_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT/train_generated_2"

correct_idx = []
total_count = 0

image_idx_counter = 0

processed_data = []

files = os.listdir(SAT_TRAIN_GENERATED_FOLDER)
files = [f for f in files if f.endswith(".jsonl")]

total_count = len(files)
for filename in tqdm(files):
    idx = int(filename.split("_")[-1].split(".")[0])
    file_path = os.path.join(SAT_TRAIN_GENERATED_FOLDER, filename)
    with open(file_path, "r") as f:
        data = []
        for line in f:
            data.append(json.loads(line))

        try:
            answer_extracted = data[-1]["response"].split("[Answer]")[1].strip()
            answer_extracted = answer_extracted.split(".")[0].strip()
        except:
            continue

    
        try:
            pred_index = ord(answer_extracted) - ord('A')
        except:
            print(answer_extracted)
            continue

        if pred_index < 0 or pred_index >= len(ds[idx]["answer_choices"]):
            continue
        answer = ds[idx]["answer_choices"][pred_index]
        if answer == ds[idx]["correct_answer"]:
            correct_idx.append(idx)
            for item in data:
                processed_data.append(parse_image_data(item))
                # processed_data.append(item)
            
print(len(correct_idx)/total_count)
print(len(correct_idx))
print(total_count)
        
            

  0%|          | 0/591 [00:00<?, ?it/s]

Neither above nor below


  4%|▎         | 21/591 [00:03<01:26,  6.61it/s]

Neither A nor B


  4%|▍         | 26/591 [00:04<01:48,  5.21it/s]

...

Cannot answer


 99%|█████████▊| 583/591 [02:10<00:01,  6.40it/s]

Neither A nor B


100%|██████████| 591/591 [02:12<00:00,  4.48it/s]

0.6277495769881557
371
591


In [7]:
print(processed_data[0])

{'messages': [{'role': 'user', 'content': [{'type': 'image', 'image': '/ocean/projects/cis250208p/shared/datasets/SAT/train_generated/images/image_0.png'}, {'type': 'text', 'text': "<image>\n### Situation Description\nGiven an image and a spatial reasoning question, we need to all entities that are included in the question.\n\n# Example 1\n[Question] You are standing at the airplane's position, facing where it is facing. Is the the person on your left or right?\n[Detect] [airplane, person]\n\n# Examples 2\n[Question] From the old man's perspective, is the person wearing a hat on the left of the green car?\n[Detect] [old man, person wearing a hat, green car]\n\n# Examples 3\n[Question] From the car's perspective, which is on the right side: the person or the tree?\n[Detect] [car, person, tree]\n\n### Your Task\nNow, given the question below, please identify the entities that are included in the question.\n\n[Question] From the camera's point of view, Considering the relative positions, 

In [10]:
# Unsloth Format
def unsloth_format(processed_data):
    output_data = []

    conv_type_counter = {
    }

    for item in processed_data:
        
        if item["conv_type"] not in conv_type_counter:
            conv_type_counter[item["conv_type"]] = 0
        conv_type_counter[item["conv_type"]] += 1

        entry = {
            "messages": [],
        }

        for msg in item["messages"]:
            e = {
                "role": msg["role"],
                "content": []
            }
            for content in msg["content"]:
                if content["type"] == "image":
                    e["content"].append({
                        "type": "image",
                        "image": content["image"]
                    })
                elif content["type"] == "text":
                    e["content"].append({
                        "type": "text",
                        "text": content["text"]
                    })
                else:
                    print("Unknown content type")

            if len(e["content"]) == 2:
                temp = e["content"][0]
                e["content"][0] = e["content"][1]
                e["content"][1] = temp

            entry["messages"].append(e)

        entry["messages"].append({
            "role": "assistant",
            "content": [{
                "type": "text",
                "text": item["response"]
            }]
        })

        output_data.append(entry)
    print(conv_type_counter)

    return output_data, conv_type_counter

In [11]:
# SpareGPT format
def sparegpt_format(processed_data):

    output_data = []

    conv_type_counter = {
    }

    for item in processed_data:
        
        if item["conv_type"] not in conv_type_counter:
            conv_type_counter[item["conv_type"]] = 0
        conv_type_counter[item["conv_type"]] += 1

        entry = {
            "messages": [],
            "images": [],
        }

        for msg in item["messages"]:
            if msg["role"] == "user":
                for sub_item in msg["content"]:
                    if sub_item["type"] == "image":
                        entry["images"].append(sub_item["image"])
                    elif sub_item["type"] == "text":
                        entry["messages"].append({
                            "content": sub_item["text"],
                            "role": "user"
                        })
                    else:
                        print("Unknown message type")
            elif msg["role"] == "system":
                if len(msg["content"]) != 1:
                    print("ERROR")

                entry["messages"].append({
                    "content": msg["content"][0]["text"],
                    "role": "assistant"
                })
            else:
                print("Unknown message role")
        
        entry["messages"].append({
            "content": item["response"],
            "role": "assistant"
        })
            
        output_data.append(entry)

    print(conv_type_counter)

    return output_data, conv_type_counter

In [17]:
output_data, conv_type_counter = unsloth_format(processed_data)

with open(SAT_TRAIN_GENERATED_FOLDER + "/train_data.json", "w") as f:
    json.dump(output_data, f, indent=2)

{'get_objects_of_interest': 371, 'detection_refinement': 771, 'get_reference_viewer': 371, 'convert_to_ego': 371, 'perspective_visual': 371, 'abstract_to_real': 371}


In [50]:
print(image_idx_counter)

3663


In [18]:
SAT_DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT"
DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets"
# with open(os.path.join(SAT_DATASET_FOLDER, "data.json"), "w") as f:

merge_data = []

with open(os.path.join(SAT_DATASET_FOLDER, "train_generated/train_data.json"), "r") as f:
    merge_data.extend(json.load(f))

with open(os.path.join(SAT_DATASET_FOLDER, "train_generated_1/train_data.json"), "r") as f:
    merge_data.extend(json.load(f))

with open(os.path.join(SAT_DATASET_FOLDER, "train_generated_2/train_data.json"), "r") as f:
    merge_data.extend(json.load(f))

with open(os.path.join(DATASET_FOLDER, "data.json"), "w") as f:
    json.dump(merge_data, f, indent=2)

In [25]:
with open(os.path.join(DATASET_FOLDER, "SAT_processed/data.json"), "r") as f:
    data = json.load(f)

with open(os.path.join(DATASET_FOLDER, "SAT_processed/test.json"), "w") as f:
    json.dump(data[0], f, indent=2)

In [2]:
import random
import json

with open("/ocean/projects/cis250208p/shared/datasets/data.json", "r") as f:
    data = json.load(f)

# shuffle data
random.shuffle(data)

# split data into train and test
train_data = data[:int(len(data)*0.9)]
test_data = data[int(len(data)*0.9):]

with open("/ocean/projects/cis250208p/shared/datasets/train_data.json", "w") as f:
    json.dump(train_data, f, indent=2)

with open("/ocean/projects/cis250208p/shared/datasets/test_data.json", "w") as f:
    json.dump(test_data, f, indent=2)
